In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
spark

In [4]:
sc = spark.sparkContext

In [5]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [36]:
# sc.stop()

# Задача

По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) посчитать агрегированную статистику по ним.

Задание необходимо сделать, используя RDD.

# Описание

Имеются следующие входные данные:

Таблица users x movies с рейтингами. Архив с датасетом нужно скачать с сайта GroupLens. Также он загружен на HDFS в /labs/laba01/ml-100k. Файл u.data содержит все оценки, а файл u.item — список всех фильмов.
Информация о структуре датасета расположена здесь.
!hdfs dfs -ls /labs/laba01/ml-100k

id фильма для расчета индивидуальных характеристик — в Личном кабинете на странице Лабы 1.

# Ознакомление с данными.

In [6]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


## Список фильмов.

In [74]:
rdd = sc.textFile("/labs/laba01/ml-100k/u.item")

In [75]:
item = rdd.map(lambda x: x.split("|")).cache()

In [76]:
item.take(1)

[['1',
  'Toy Story (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)',
  '0',
  '0',
  '0',
  '1',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0']]

In [77]:
item.filter(lambda x: x[0] == "302").take(2)

[['302',
  'L.A. Confidential (1997)',
  '01-Jan-1997',
  '',
  'http://us.imdb.com/M/title-exact?L%2EA%2E+Confidential+(1997)',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '1',
  '0',
  '0',
  '1',
  '0',
  '0']]

## Оценки пользователей.

In [31]:
# rdd = spark.read.option("delimiter", "\t").csv("/labs/laba01/ml-100k/u.data")
rdd = sc.textFile("/labs/laba01/ml-100k/u.data")
data = rdd.map(lambda x: x.split("\t")).cache()

In [64]:
data.take(1)

[['196', '242', '3', '881250949']]

In [79]:
hist_film = list(data.filter(lambda x: x[1] == "302").map(lambda x: (x[2], 0)).sortByKey().countByKey().values())

In [80]:
hist_film

[2, 10, 46, 119, 120]

In [81]:
hist_all = list(data.map(lambda x: (x[2], 0)).sortByKey().countByKey().values())

In [82]:
hist_all

[6110, 11370, 27145, 34174, 21201]

In [83]:
df = dict(zip(("hist_film", "hist_all"),(hist_film, hist_all)))

In [84]:
df

{'hist_film': [2, 10, 46, 119, 120],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [85]:
with open("lab01.json", "w") as write_file:
    json.dump(df, write_file)